In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans as Clst
from sklearn.preprocessing import MinMaxScaler as MX

In [ ]:
df = pd.read_csv('./merged-data.csv')
df.replace(9999.99, 0, inplace=True)
df.head()

In [ ]:
scaler = MX()
df[['bx_gsm']] = scaler.fit_transform(df[['bx_gsm']])
df[['by_gsm']] = scaler.fit_transform(df[['by_gsm']])
df[['bz_gsm']] = scaler.fit_transform(df[['bz_gsm']])
df[['bt']] = scaler.fit_transform(df[['bt']])
df[['intensity']] = scaler.fit_transform(df[['intensity']])
df[['declination']] = scaler.fit_transform(df[['declination']])
df[['inclination']] = scaler.fit_transform(df[['inclination']])
df[['north']] = scaler.fit_transform(df[['north']])
df[['east']] = scaler.fit_transform(df[['east']])
df[['vertical']] = scaler.fit_transform(df[['vertical']])
df[['horizontal']] = scaler.fit_transform(df[['horizontal']])
df.head()

In [ ]:
krng = range(1,13)
sse = []
for i in krng:
    clusterer = Clst(n_clusters=i)
    clusterer.fit(df[df.columns.values])
    sse.append(clusterer.inertia_)


In [ ]:
for err in sse:
    print(err)
print()
print("min:",np.min(sse))

In [ ]:
plt.plot(krng,sse)
plt.xlabel('sum of squared err')
plt.ylabel('k')

In [ ]:
clusterer = Clst(n_clusters=4)
y_predicted = clusterer.fit_predict(df[df.columns.values])
df['class'] = y_predicted

In [ ]:
df.head()

In [ ]:
# check distribution
df['class'].value_counts()

In [ ]:
grouped = df.groupby('class')

result = grouped.agg({'bx_gsm': ['mean', 'median'], 'by_gsm': ['mean', 'median'],
                       'bz_gsm': ['mean', 'median'], 'bt': ['mean', 'median'],
                       'vertical':['mean','median']})
# Rename the columns for clarity
result.columns = ['bx_mean', 'bx_median', 'by_mean',
                   'by_median', 'bz_mean', 'bz_median',
                     'bt_mean', 'bt_median','vt_mean','vt_median']
# Reset the index to make 'class'
result.reset_index(inplace=True)


In [ ]:
print(result)

In [ ]:
result['sum_of_medians'] = result['bx_median'] + result['by_median'] + \
    result['bz_median'] + result['bt_median'] + result['vt_median']

# Group by 'class' and calculate the sum
sum_of_medians = result.groupby('class')['sum_of_medians'].sum().reset_index()

sum_of_medians

In [ ]:
new_df = pd.read_csv('./merged-data.csv')
new_df.replace(9999.99, np.nan, inplace=True)
new_df['class'] = y_predicted
new_df.to_csv('classification.csv')